# Fine-grained Parsing

After initial parsing, let's parse out details from each resume section individually

In [55]:
import json
from pprint import pprint
from ollama import chat
from pydantic import BaseModel, RootModel, Field

from utils.with_structured_output import with_structured_output

In [56]:
with open("output/parsed_resume.json", "r") as file:
    parsed_resume = json.load(file)

In [57]:
parsed_resume

{'Experience': 'Honeywell FM&T, Kansas City National Security Campus – Kansas City, MO June 2024 – August 2024\nElectrical Engineering Intern II (Purchase Product Engineer in PCBs)\n• Assessed PCB drawings and managed external stakeholders to improve manufacturability of complex circuit designs.\n• Collaborated with numerous internal agencies/teams to design, manufacture, and buy critical electrical components.\n• Standardized process and documentation of scrapping parts, ensuring audit compliance and saving Product Engineers 48\neffort hours annually.\n• Automated solder void detection in bottom-terminated components using Matlab to ensure strict IPC compliance.\nTURTLE Robotics, Officer - College Station, TX August 2023 – Present\nTreasurer September 2024 – Present\n• Coordinating with Mechanical Engineering department to purchase supplies and manage $15,000 budget.\n• Supervising payment system for membership dues to maintain timely collection and accurate records.\n• Facilitating s

## Parse Education

In [58]:
class School(BaseModel):
    name: str           = Field(..., alias="Name")
    majors: list[str]   = Field(..., alias="Majors")
    minors: list[str]   = Field(..., alias="Minors")
    gpa: float          = Field(None, alias="GPA")
    grad_year: int      = Field(..., alias="Graduation Year")

class Education(RootModel[list[School]]):
    pass

In [59]:
EDUCATION_EXTRACTION_PROMPT = """
You are an expert resume parser. Given some resume text, your job is to parse education information as a list of JSON objects representing each school attended. Follow this format for each school:
    {{
        "Name": "<Name of School>",
        "Majors": ["list", "of", "majors"],
        "Minors": ["list", "of", "minors"],
        "GPA": <GPA>,
        "Graduation Year": <Graduation Year>
    }},

Notes:
1. If there are no minors, set "Minors" to an empty list.
2. If there is no GPA listed, set "GPA" to None.
3. If any school does not have a graduation year listed, omit the school from the output.
4. Output the full name of all degrees, e.g., "BS in Computer Science", "M.S. in Information Science". Note that the resume may contain a double major. If so, output all degrees with their full names, making sure to incldue the type of degree for each major ("BS," "MS," etc.). Please note that some schools offer emphasis areas or modifiers to the major that are not themselves considered majors, e.g. "Computer Science with statistics emphasis" is equivalent to "Computer Science."
5. If the resume does not contain information for one of the sections, return an empty list for that section.

Extracted information must be **explicitly contained in the resume.**

Resume text:
{resume_text}

Output:
"""

In [60]:
education_info = with_structured_output(
    prompt=EDUCATION_EXTRACTION_PROMPT.format(resume_text=parsed_resume["Education"]),
    schema=Education)

In [61]:
education_info

[{'Name': 'Texas A&M University',
  'Majors': ['BS in Electrical Engineering'],
  'Minors': ['Minor in Chinese Program'],
  'Graduation Year': 2025,
  'GPA': 3.0}]

## Parse Experience

In [62]:
class Experience(BaseModel):
    company: str = Field(..., alias="Company")
    role: str = Field(..., alias="Role")
    contributions: list[str] = Field(..., alias="Contributions")

In [63]:
EXPERIENCE_EXTRACTION_PROMPT = """
You are an expert at parsing resumes. Given some resume text, your job is to extract information about the candidate's work experience and format it as a list of JSON objects, where each object has the following format:
    {{
        "Company": "<Company>",
        "Role": "<Applicant's role at the company>",
        "Contributions": ["list", "of", "contributions", "in", "the", "role"]
    }}
    
The extracted information must be **explicitly contained in the resume.**

Resume text:
{resume_text}

Output:
"""

In [64]:
experience_info = with_structured_output(
    EXPERIENCE_EXTRACTION_PROMPT.format(resume_text=parsed_resume["Experience"]),
    RootModel[list[Experience]])

In [65]:
experience_info

[{'Company': 'Honeywell FM&T',
  'Role': 'Electrical Engineering Intern II (Purchase Product Engineer in PCBs)',
  'Contributions': ['Assessed PCB drawings and managed external stakeholders to improve manufacturability of complex circuit designs.',
   'Collaborated with numerous internal agencies/teams to design, manufacture, and buy critical electrical components.',
   'Standardized process and documentation of scrapping parts, ensuring audit compliance and saving Product Engineers 48 effort hours annually.',
   'Automated solder void detection in bottom-terminated components using Matlab to ensure strict IPC compliance.']},
 {'Company': 'TURTLE Robotics',
  'Role': 'Officer',
  'Contributions': ['Coordinating with Mechanical Engineering department to purchase supplies and manage $15,000 budget.',
   'Supervising payment system for membership dues to maintain timely collection and accurate records.',
   'Facilitating supply reimbursement process for 11 project teams to improve efficie

## Skill Parsing 

In [66]:
class Skills(BaseModel):
    technical_skills: list[str] = Field(..., alias="Technical Skills")
    domain_specific_skills: list[str] = Field(..., alias="Domain-Specific Skills")

In [67]:
SKILL_EXTRACTION_TEMPLATE = """
You are an expert are parsing skills from resumes.

Given resume text, please parse individual technical (e.g., programming languages, tools, frameworks, databases) and domain-specific (e.g., methodologies, architectures, or specialized techniques) skills contained in the resume. **Ensure that you do not miss any domain-specific skills.**

Format your output as a JSON object as follows:
    {{
        "Technical Skills": ["list", "of", "technical", "skills"]
        "Domain-Specific Skills": ["list", "of" "domain", "specific", "skills"]
    }}

Resume text:
```
{resume_text}
```

Parsed skills:
"""

In [68]:
skills_info = with_structured_output(
    prompt=SKILL_EXTRACTION_TEMPLATE.format(
        resume_text=parsed_resume["Experience"]
            + parsed_resume["Projects"]
            + parsed_resume["Skills"]
            + parsed_resume["Research"]
            + parsed_resume["Leadership"]),
    schema=Skills)

In [69]:
skills_info

{'Technical Skills': ['Matlab',
  'SystemVerilog',
  'Python',
  'C++',
  'Multisim',
  'Xilinx Vivado',
  'LTspice',
  'CAM350',
  'Microsoft Office (Word, Excel, PowerPoint)',
  'Raspberry Pi',
  'Arduino',
  'oscilloscope',
  'signal generator',
  'multimeter',
  '3D printer'],
 'Domain-Specific Skills': ['PCB design and manufacturability',
  'Automated solder void detection',
  'IPC compliance',
  'Turret robot development (Xilinx FPGA-driven)',
  'Computer vision system (Vitis HLS with C++)',
  'Camera chassis and drive base design',
  'Robotics automation (lighting, water, and nutrient delivery)',
  'LED grow light system (Raspberry Pi-controlled)',
  'Digital circuit design (ALUs, logic gates, adders)',
  'SystemVerilog programming',
  'FPGA simulation (Xilinx Zynq series)',
  'Traffic light controller design',
  'Combination lock systems design',
  '4-bit addition/subtraction calculator design']}

### Putting it all together

In [70]:
parsed_info = {
    "Education": education_info,
    "Experience": experience_info,
    "Skills": skills_info
}
pprint(parsed_info)

{'Education': [{'GPA': 3.0,
                'Graduation Year': 2025,
                'Majors': ['BS in Electrical Engineering'],
                'Minors': ['Minor in Chinese Program'],
                'Name': 'Texas A&M University'}],
 'Experience': [{'Company': 'Honeywell FM&T',
                 'Contributions': ['Assessed PCB drawings and managed external '
                                   'stakeholders to improve manufacturability '
                                   'of complex circuit designs.',
                                   'Collaborated with numerous internal '
                                   'agencies/teams to design, manufacture, and '
                                   'buy critical electrical components.',
                                   'Standardized process and documentation of '
                                   'scrapping parts, ensuring audit compliance '
                                   'and saving Product Engineers 48 effort '
                         

In [71]:
with open("output/parsed_info.json", "w") as file:
    json.dump(parsed_info, file, indent=4)